# CHIKV_Mac HIPPO FFF Prod. 11 (rgen redo)

## Setup

In [1]:
%load_ext autoreload
%autoreload 2
import hippo
from pathlib import Path
from mlog import setup_logger
import pandas as pd
import numpy as np
import molparse as mp
logger=setup_logger('notebook')

In [2]:
animal = hippo.HIPPO('CHIKV_prod11', 'CHIKV_prod11.sqlite', copy_from='CHIKV_prod10.sqlite', overwrite_existing=False)

Creating HIPPO animal
name = CHIKV_prod11
db_path = CHIKV_prod11.sqlite
Copying CHIKV_prod10.sqlite --> CHIKV_prod11.sqlite
DEBUG: Copied 10000 of 161313 pages...
DEBUG: Copied 20000 of 161313 pages...
DEBUG: Copied 30000 of 161313 pages...
DEBUG: Copied 40000 of 161313 pages...
DEBUG: Copied 50000 of 161313 pages...
DEBUG: Copied 60000 of 161313 pages...
DEBUG: Copied 70000 of 161313 pages...
DEBUG: Copied 80000 of 161313 pages...
DEBUG: Copied 90000 of 161313 pages...
DEBUG: Copied 100000 of 161313 pages...
DEBUG: Copied 110000 of 161313 pages...
DEBUG: Copied 120000 of 161313 pages...
DEBUG: Copied 130000 of 161313 pages...
DEBUG: Copied 140000 of 161313 pages...
DEBUG: Copied 150000 of 161313 pages...
DEBUG: Copied 160000 of 161313 pages...
DEBUG: Copied 161313 of 161313 pages...
DEBUG: hippo3.Database.__init__()
DEBUG: Database.path = CHIKV_prod11.sqlite
DEBUG: hippo3.Database.connect()
DEBUG: sqlite3.version='2.6.0'
 SUCCESS  Database connected @ CHIKV_prod11.sqlite!
 SUCCESS  In

## Create the RRG

In [3]:
recipe = hippo.Recipe.from_json(animal.db, 'CHIKV_prod8_recipes/Recipe_1G87NWP.json', allow_db_mismatch=True)

 DISK  Reading CHIKV_prod8_recipes/Recipe_1G87NWP.json
session = /opt/xchem-fragalysis-2/maxwin/CHIKV_HIPPO/hippo_prod/CHIKV_prod11.sqlite
in file = /opt/xchem-fragalysis-2/maxwin/CHIKV_HIPPO/hippo_prod/CHIKV_prod9.sqlite
 Warning  Database path mismatch
Recipe was generated at: 2024-10-02 15:15:05.325711
reactants = {I x 125}
intermediates = {I x 0}
products = {I x 234}
reactions = {R x 235}


In [ ]:
gen = hippo.RandomRecipeGenerator(db=animal.db, start_with=recipe, suppliers=['Stock', 'Enamine'])

DEBUG: RandomRecipeGenerator.__init__()
database = CHIKV_prod11.sqlite
max_lead_time = None
suppliers = ['Stock', 'Enamine']
 DISK  Writing CHIKV_prod11_recipes/
DEBUG: Solving route pool...


 28%|██▊       | 2144/7664 [00:12<00:35, 155.14it/s]

In [ ]:
gen = hippo.RandomRecipeGenerator.from_json(animal.db, "CHIKV_prod8_rgen.json")

## Check Quotes / Fingerprints

In [ ]:
animal.db.select_where(table='quote', query='AVG(quote_amount), COUNT(DISTINCT quote_compound)', key='supplier', value='Stock')

In [ ]:
animal.db.select_where(table='interaction', query='COUNT(DISTINCT interaction_pose)', key='interaction_id IS NOT NULL')

In [ ]:
animal.poses

In [ ]:
animal.db.select_where(table='pose', query='COUNT(DISTINCT pose_reference)', key='pose_id IS NOT NULL')

In [ ]:
animal.db.count('route')

In [ ]:
gen.route_pool

# Test generation

In [ ]:
r = gen.generate(shuffle=False, budget=30000, max_products=1200, max_reactions=3000, currency='EUR', debug=False)

In [40]:
r.reactants.df

,compound_id,amount,quote_id,supplier,max_lead_time,quoted_amount
0,85,60.0,10539,Enamine,None,100.0
1,4554,10.0,65,Enamine,None,20.0
2,4466,1.0,36,Enamine,None,20.0
3,4467,1.0,37,Enamine,None,20.0
4,4440,10.0,25,Enamine,None,20.0
...,...,...,...,...,...,...
181,52819,1.0,4096,"[Stock, Enamine]",None,20.0
182,52829,1.0,4098,"[Stock, Enamine]",None,20.0
183,78836,1.0,5033,"[Stock, Enamine]",None,20.0
184,78868,1.0,5040,"[Stock, Enamine]",None,20.0


## Check why impossible routes are in the pool

In [ ]:
sql = """
WITH possible_reactants AS (
			SELECT quote_compound, COUNT(CASE WHEN quote_supplier IN ("Enamine", "Stock") THEN 1 END) AS [count_valid] FROM quote
			GROUP BY quote_compound
		),

		route_reactants AS (
			SELECT route_id, route_product, COUNT(CASE WHEN count_valid = 0 THEN 1 WHEN count_valid IS NULL THEN 1 END) AS [count_unavailable] FROM route
			INNER JOIN component ON component_route = route_id
			LEFT JOIN possible_reactants ON quote_compound = component_ref
			WHERE component_type = 2
			GROUP BY route_id
		)

SELECT COUNT(route_id) FROM route_reactants
WHERE count_unavailable = 0
"""
# AND route_id = 77365

c = animal.db.execute(sql)

results = c.fetchall()
results

In [ ]:
sql = """
WITH possible_reactants AS (
			SELECT quote_compound, COUNT(CASE WHEN quote_supplier IN ("Enamine", "Stock") THEN 1 END) AS [count_valid] FROM quote
			GROUP BY quote_compound
		),

		route_reactants AS (
			SELECT route_id, route_product, count_valid, COUNT(CASE WHEN count_valid = 0 THEN 1 END) AS [count_unavailable] FROM route
			INNER JOIN component ON component_route = route_id
			LEFT JOIN possible_reactants ON quote_compound = component_ref
			WHERE component_type = 2
            GROUP BY route_id
		)

SELECT route_id FROM route_reactants
WHERE count_unavailable = 0
"""

c = animal.db.execute(sql)

results = c.fetchone()
results

In [ ]:
sql = """
WITH possible_reactants AS (
			SELECT quote_compound, COUNT(CASE WHEN quote_supplier IN ("Enamine", "Stock") THEN 1 END) AS [count_valid] FROM quote
			GROUP BY quote_compound
		),

		route_reactants AS (
			SELECT route_id, route_product, component_ref, count_valid, COUNT(CASE WHEN count_valid = 0 THEN 1 WHEN count_valid IS NULL THEN 1 END) AS [count_unavailable] FROM route
			INNER JOIN component ON component_route = route_id
			LEFT JOIN possible_reactants ON quote_compound = component_ref
			WHERE component_type = 2
            AND route_id = 77365
            GROUP BY route_id
		)

SELECT * FROM route_reactants
"""

c = animal.db.execute(sql)

results = c.fetchall()
results

In [ ]:
animal.db.execute("""
SELECT quote_compound, CASE WHEN quote_supplier IN ("Enamine", "Stock") THEN 1 END AS [count_valid] FROM quote
WHERE quote_compound IN (4574,91477)
""").fetchall()

In [ ]:
animal.db.execute("""

""").fetchall()

In [ ]:
route_ids = [r[0] for r in results]
route_products = [r[1] for r in results]
count_unavailable = [r[2] for r in results]
len(route_ids)

In [ ]:
animal.db.get_route(id=1).reactant

In [ ]:
animal.C91478.reactions[0].reactants.ids

In [ ]:
route_id = 77365
route_id in route_ids

In [ ]:
animal.C4574.get_quotes(df=True)

In [ ]:
animal.db.get_route(id=670).reactions.types

# Check missing multi-step chemistry

In [ ]:
inchi = 'YIIMVUSFSKUYCA-UHFFFAOYSA-N'
inchi = 'XXPRWRCNORVHSY-UHFFFAOYSA-N'

In [ ]:
c = animal.compounds[inchi]
c

In [ ]:
recipes = c.get_recipes(quoted_only=False)
len(recipes)

In [ ]:
recipe = recipes[0]

In [ ]:
recipe.sankey()

In [ ]:
recipe.reactions.summary()

In [ ]:
animal.C4449.get_quotes(df=True)

In [ ]:
animal.C4497.get_quotes(df=True)

In [ ]:
animal.C4499.get_quotes(df=True)

In [ ]:
animal.db.select_all_where(table='quote', key='entry', value='EN300-41185')

In [ ]:
animal.C4499.draw()

In [ ]:
animal.C4499.smiles

In [ ]:
animal.compounds(smiles='CNCC(N)=O.HCl')

In [ ]:
c.draw()

In [ ]:
c.poses.interactive()

In [ ]:
animal.P6264.render()

In [ ]:
animal.P6264.draw()

In [ ]:
recipe.reactions